In [34]:
import os
import json
import random
import sys

Lisanüanss: lõplik jaotus peaks olema selline, et seal leidub üks alamosa, kus ei ole ühtegi käsitsi morf ühestatud protokolli. Käsitsi morf ühestatud protokollide nimekiri on siin:

In [59]:
def read_protocols_tagged_by_hand(file = "k2sitsi_morfiga_protokollid.txt"):
    protocols_tagged_by_hand = []
    with open(file, 'r', encoding="UTF-8") as f:
        lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "")
        if line != "" and (line != "k2sitsi_morfiga_protokollid2:" and line != "k2sitsi_morfiga_protokollid1:"):
            line += ".json"
            protocols_tagged_by_hand.append(line)
            
    return protocols_tagged_by_hand

Ma arvan, et "ideaalseks jaotuseks" võibki esialgu võtta nimeüksuste jaotuse liikidesse kogu korpusel, mille annab skript "check_ner_layers.ipynb". 

In [36]:
def calculate_whole_corpus_distribution(filenames = os.listdir("vallakohtufailid_json/")):
    all_annotations = []
    for file in filenames:
        with open("vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                all_annotations.append(dictionary.get('annotations')[0].get('nertag'))
    per = all_annotations.count("PER") / len(all_annotations) * 100
    loc_org = all_annotations.count("LOC_ORG") / len(all_annotations) * 100
    loc = all_annotations.count("LOC") / len(all_annotations) * 100
    org = all_annotations.count("ORG") / len(all_annotations) * 100
    misc = all_annotations.count("MISC") / len(all_annotations) * 100

    ideal_distribution = {'PER': per, 'LOC_ORG' : loc_org, 'LOC' : loc, 'ORG' : org, 'MISC' : misc}
    
    return ideal_distribution

Kõigepealt tekita sõnastik, kus on iga json faili kohta nimeüksuste statistika liikide kaupa "gold_ner" kihist

In [37]:
def find_document_ne_statistics(filenames = os.listdir("vallakohtufailid_json/")):
    statistics = dict()
    for file in filenames:
        ner_annotations = []
        with open("vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                ner_annotations.append(dictionary.get('annotations')[0].get('nertag'))

        statistics_for_file = dict()
        for annotation in set(ner_annotations):
            statistics_for_file[str(annotation)] = ner_annotations.count(annotation)
        statistics[file] = statistics_for_file

    return statistics

Statistics, the ideal distribution and the protocols tagged by hand:

In [60]:
statistics = find_document_ne_statistics()
ideal_distribution = calculate_whole_corpus_distribution()
protocols_tagged_by_hand = read_protocols_tagged_by_hand()

In [39]:
def swap_items_in_lists(A, B, i, j):
    temp_b = B[j]
    B[j] = A[i]
    A[i] = temp_b
    return A, B

def improve_scores(largest, second_largest):
    score_largest_old = calculate_score(largest)
    score_second_largest_old = calculate_score(second_largest)
    for i in range(len(largest)):
        for j in range(len(second_largest)):
            largest, second_largest = swap_items_in_lists(largest, second_largest, i, j)
            
            score_largest_new = calculate_score(largest)
            score_second_largest_new = calculate_score(second_largest)

            if score_largest_old > score_largest_new and score_second_largest_old > score_second_largest_new:
                print(f"{i, j} Skoor paranes mõlema alamkorpuse lõikes.")
                score_largest_old = score_largest_new
                score_second_largest_old = score_second_largest_new
                i += 1
            elif score_largest_old < score_largest_new and score_second_largest_old > score_second_largest_new:
                print(f"{i, j} Skoor paranes ainult esimeses massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
            elif score_largest_old > score_largest_new and score_second_largest_old < score_second_largest_new:
                print(f"{i, j} Skoor paranes ainult teises massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
            else:
                print(f"{i, j} Skoor ei paranenud kummaski massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
    return largest, second_largest

In [155]:
def get_rid_of_hand_tagged(least_hand_tagged, other_files):
    for i, x in enumerate(set(least_hand_tagged).intersection(protocols_tagged_by_hand)):
        print(i+1, x)
        score_old = calculate_score(least_hand_tagged)

        for files in other_files:
            breaking = "no"
            score_old_output = calculate_score(files)

            for j, y in enumerate(files):
                if y in protocols_tagged_by_hand:
                    continue
                else:
                    swap_items_in_lists(least_hand_tagged, files, i, j)
                    score_new = calculate_score(least_hand_tagged)
                    score_new_output = calculate_score(files)
                    if score_new < score_old and score_new_output < score_old_output:
                        print("Leidsin vaste! Vahetasin failiga", y)
                        breaking = "yes"
                        break
                    elif score_new == score_old and score_new_output == score_old_output or \
                         abs(score_new - score_old) < 0.05 and abs(score_new_output-score_old_output) < 0.05:
                        print("Leidsin vaste! Vahetasin failiga", y)
                        breaking = "yes"
                        break
                    else:
                        swap_items_in_lists(files, least_hand_tagged, i, j)
                        continue
            if breaking == "yes":
                break
    return least_hand_tagged, other_files

Seejärel loo funktsioon, mis leiab (eelmise sõnastiku abil) nimeüksuste proportsionaalse jaotuse etteantud alamkorpuses. Sisendiks on alamkorpus ehk järjend json failide nimedest ja tulemusena tagastatakse sõnastik, kus on iga nimeüksuse liigi kohta toodud selle osakaal alamkorpuses: { 'PER': x, 'LOC': y, ... }. Osakaal on siis jagatis: {nimeüksuseid antud liigist} / {kõik nimeüksused alamkorpuses} -- kas protsendina või arvuna vahemikust 0.0 kuni 1.0, kuidas järgnevatel etappidel mugavam tundub.

In [41]:
def calculate_proportions(filenames):
    statistics_for_proportion = {}
    
    for file in filenames:
        statistics_for_proportion[file] = statistics[file]

    all_annotations = list()
    for item in statistics_for_proportion.values():
        for key in item:
            for i in range(item[key] + 1):
                all_annotations.append(key)
                
    proportions = dict()
    for file in statistics_for_proportion:
        for key in statistics_for_proportion[file].keys():
            proportion = all_annotations.count(key) / len(all_annotations) * 100
            proportions[key] = proportion
        
    return proportions

Neljas funktsioon: juhujaotuse genereerimine. Jagab sisendiks antud korpuse dokumendid (järjend json failide nimedest) N juhuslikuks alamosaks -- nii, et iga alamosa koosneb sellistest dokumentidest, mida mujal alamosades ei ole ning kogu korpuse dokumendid on kaetud. Selleks: a) järjesta kõik dokumendid juhuslikult random.sample funktsiooni abil (et eksperiment oleks korratav, fikseeri enne iga juhuslikku järjestamist random.seed mingi kindla väärtusega). b) jaga järjestus N alamosaks nii, et igas alamosas on enam-vähem võrdne arv dokumente. Selleks on mitmeid viise (vt nt siit), aga oluline on, et viimane alamosa ei tuleks liiga väike. Tagasta tulemus (N liikmeline list, mille iga liige on omakorda dokumentide list);

In [42]:
def n_even_chunks(filenames, n):
    files = []
    last = 0
    for i in range(1, n+1):
        current = int(round(i* (len(filenames) / n)))
        files.append(filenames[last:current])
        last = current
    return files

def generate_random_division(filenames, n):
    random_distributions = []
    for i in range(n):
        filenames = random.sample(filenames, len(filenames))
        random_distributions.append(filenames)
    
    return random_distributions

Kolmas funktsioon: saab sisendiks alamkorpuse (järjendi json failide nimedest) ja arvutab sellele skoori, mis näitab seda, kui lähedal on nimeüksuste proportsionaalne jaotus nö ideaalsele jaotusele. Algoritm võib lihtsalt olla selline, et leiab iga nimeüksuste tüübi kohta, mitu protsendipunkti erineb osakaal antud alamkorpuses nimeüksuse liigi osakaalust selles nö ideaalses jaotuses, ja summeerib erinevused. Skoor töötab siis nii, et mida väiksem erinevuste summa, seda parem.

In [43]:
def calculate_score(filenames):
    proportions = calculate_proportions(filenames)
    score = 0
    for proportion in proportions:
        ideal_distribution_proportion = ideal_distribution[proportion]
        current_proportion = proportions[proportion]
        
        if current_proportion == ideal_distribution_proportion:
            score += 0
        else:
            score += abs(ideal_distribution_proportion - current_proportion)
    
    return score

In [157]:
seed = 0
random.seed(seed)
print("Seed:", seed)

subdistributions = list()
random_distributions = generate_random_division(sorted(os.listdir("vallakohtufailid_json/")), 4)
for distribution in random_distributions:
    distribution = n_even_chunks(distribution, 6)
    subdistributions.append(distribution)
    
sums_of_distributions = list()
list_of_scores = list()

for distribution in subdistributions:
    scores = dict()
    for files in distribution:
        score = calculate_score(files)
        scores[score] = files
        print(score)
    list_of_scores.append(scores)
    print(f"Score of sub-corpus: {sum(scores.keys()) / len(scores.keys())}")
    print(" ")

for listofscores in list_of_scores:    
    sums_of_distributions.append(sum(listofscores.keys()) / len(listofscores.keys()))

Seed: 0
7.49810704503649
9.918086072691803
7.5555920651177555
6.59971016626813
11.428219874925578
8.667367929638713
Score of sub-corpus: 8.61118052561308
 
10.603654082464963
7.5346799108846385
6.602481512183512
10.360863129648
9.178850105842189
7.332724881744491
Score of sub-corpus: 8.602208937127966
 
8.752548402461386
7.403612076075569
8.406346014189152
8.58044886933627
8.758484246007118
9.385208091838054
Score of sub-corpus: 8.547774616651258
 
6.8019504097935535
7.676950409793554
11.904403512246654
11.547089541240748
6.6362453555586
7.148161850344615
Score of sub-corpus: 8.619133513162954
 


1. Kõigepealt genereeri mingi arv korpuse juhuslikke jaotuseid 6-ks alamosaks. Nt N=10 tükki.

2. Leia iga jaotuse skoor. Jaotuse skoor = leia kõigi kuue alamosa skoorid skoorimise funktsiooni abil ning liida kokku. Kui peaks juhtuma, et mõnele jaotusele trehvabki kohe väga väike skoor (nt kogu-erinevus <= 5 protsendipunkti), siis tasub protsess seisma panna ja tulemustele peale vaadata, st äkki ongi sobiv jaotus käes. Aga kui nii ei lähe, siis edasi:

3. Vali välja kõige kõrgema skoori saanud jaotus.

4. Iteratiivselt: 4.1) Leia jaotusest kaks alamosa, mille erinevus nö ideaalsest jaotusest on kõige suurem, 4.2) paarita mõlema jaotuse kõiki dokumente omavahel (topelt-tsükkel) ja leia iga paari (x1, y1) puhul, milliseks muutub mõlema alamosa skoor, kui vahetada dokumendid alamosades. 4.3) Teosta ümberpaigutus, mis parandas skoori kõige rohkem. 4.4) Leia uuesti kogu jaotuse skoor; kui see on väga väike (nt kogu-erinevus <= 5 protsendipunkti), siis seiska protsess ja vaatame tulemustele peale. Vastasel juhul pöördu tagasi punkti 4.1.

In [ ]:
smallest_score = list_of_scores[sums_of_distributions.index(min(sums_of_distributions))]


while (sum(smallest_score.keys()) / len(smallest_score.keys())) >= 5:
    keys_list = sorted(smallest_score.keys())
    largest_key = keys_list[-1]
    second_largest_key = keys_list[-2]
    
    largest = smallest_score[largest_key]
    second_largest = smallest_score[second_largest_key]
    
    print(f"Algsed skoorid on {keys_list[-1]} ning {keys_list[-2]}.")
    largest, second_largest = improve_scores(largest, second_largest)
    score_after_improving_largest = calculate_score(largest)
    score_after_improving_second_largest = calculate_score(second_largest)
    print(f"Skoorid pärast parandamist on {score_after_improving_largest} ning {score_after_improving_second_largest}.")
    del smallest_score[largest_key]
    del smallest_score[second_largest_key]
    smallest_score[score_after_improving_largest] = largest
    smallest_score[score_after_improving_second_largest] = second_largest

In [158]:
smallest_score = list_of_scores[sums_of_distributions.index(min(sums_of_distributions))]

final_files_distribution = []

number_of_hand_tagged_protocols = {}
for files in smallest_score.values():
    number_of_hand_tagged_protocols[len(set(files).intersection(protocols_tagged_by_hand))] = files

least_hand_tagged_old = number_of_hand_tagged_protocols[(min(number_of_hand_tagged_protocols.keys()))]

least_hand_tagged, other_files = get_rid_of_hand_tagged(least_hand_tagged_old, number_of_hand_tagged_protocols.values())

print(len(set(least_hand_tagged).intersection(protocols_tagged_by_hand))) #SIIN ERROR (?)

final_files.append(least_hand_tagged)
for files in other_files:
    final_files.append(files)

1 L22ne_Martna_Martna_id14205_1869a.json
Leidsin vaste! Vahetasin failiga V6ru_P6lva_K2hri_id21539_1850a.json
2 Viljandi_Pilistvere_K6o_id24950_1843a.json
Leidsin vaste! Vahetasin failiga Tartu_V6nnu_Ahja_id16984_1884a.json
3 Tartu_R6ngu_Aakre_id12559_1827a.json
Leidsin vaste! Vahetasin failiga Tartu_V6nnu_Ahja_id17632_1885a.json
4 Tartu_V6nnu_Kiidj2rve_id24727_1866a.json
Leidsin vaste! Vahetasin failiga Harju_Kose_Kose-Uuem6isa_id10389_1870a.json
5 Viru_Haljala_Vihula_id5807_1876a.json
6 Harju_Kose_Kose-Uuem6isa_id5292_1869a.json
Leidsin vaste! Vahetasin failiga Tartu_V6nnu_Ahja_id18911_1887a.json
7
